# 02 Training - Conv

#### Cognibit Solutions LLP

Convolutional model training notebook. (Baseline Approach)

In [1]:
import sys
import os
import tensorflow as tf

sys.path.append("../libs")
from classification import input_data
from classification import models
from classification import trainer
from classification import freeze

W0906 00:53:47.095567 12612 deprecation_wrapper.py:119] From ../libs\classification\trainer.py:17: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0906 00:53:47.111188 12612 deprecation_wrapper.py:119] From ../libs\classification\trainer.py:17: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



Change the data folder to use the required data folder.

### Flags

In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\data\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',30,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',10.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC fingerprint')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

### Variables

In [4]:
model_architecture='conv'
start_checkpoint=None
logging_interval=1
eval_step_interval=2000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.1
learning_rate='0.001,0.0001' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='15000,300' #Declare  the training steps for which the learning rates will be used
batch_size=100

### Initialise the get_train_data() get_val_data() and get_test_data() Function

In [6]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [7]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [8]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings)

..\data\raw\train\audio\*\*.wav


W0906 01:01:39.199846 12612 deprecation_wrapper.py:119] From ../libs\classification\input_data.py:319: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0906 01:01:39.260841 12612 deprecation_wrapper.py:119] From ../libs\classification\input_data.py:320: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\data\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\data\raw\train\audio\_background_noise_\dude_miaowing.wav
..\data\raw\train\audio\_background_noise_\exercise_bike.wav
..\data\raw\train\audio\_background_noise_\pink_noise.wav
..\data\raw\train\audio\_background_noise_\running_tap.wav
..\data\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 98, 257), dtype=float32)


In [9]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings,background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_fingerprints,train_ground_truth

In [10]:
def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_fingerprints,validation_ground_truth

### Training

In [11]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    fingerprint_size = model_settings['fingerprint_size']
    label_count = model_settings['label_count']
    fingerprint_input = tf.placeholder(
      tf.float32, [None, fingerprint_size], name='fingerprint_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    # Create Model
    
    logits, dropout_prob = models.create_model(
      fingerprint_input,
      model_settings,
      model_architecture,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,fingerprint_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [ ]:
tf.app.run(main=main)

W0906 01:01:54.842876 12612 deprecation_wrapper.py:119] From ../libs\classification\models.py:329: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0906 01:01:54.860822 12612 deprecation.py:506] From ../libs\classification\models.py:337: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0906 01:01:54.872755 12612 deprecation_wrapper.py:119] From ../libs\classification\models.py:340: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0906 01:01:54.904707 12612 deprecation.py:323] From ../libs\classification\trainer.py:60: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to f

I0906 01:03:08.008466 12612 trainer.py:129] Step #25: rate 0.001000, accuracy 20.0%, cross entropy 2.289561
I0906 01:03:08.008466 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-25"
I0906 01:03:10.605964 12612 trainer.py:129] Step #26: rate 0.001000, accuracy 24.0%, cross entropy 2.262996
I0906 01:03:10.605964 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-26"
I0906 01:03:13.317946 12612 trainer.py:129] Step #27: rate 0.001000, accuracy 19.0%, cross entropy 2.391899
I0906 01:03:13.317946 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-27"
I0906 01:03:15.954809 12612 trainer.py:129] Step #28: rate 0.001000, accuracy 14.0%, cross entropy 2.356706
I0906 01:03:15.954809 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-28"
I0906 01:03:18.747437 12612 trainer.py:129] Step #29: rate 0.001000, accuracy 22.0%, cross entropy 2.211111
I0906 01:03:18.747437 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-29"
I0906 01:03:21.38693

I0906 01:05:04.696127 12612 trainer.py:129] Step #67: rate 0.001000, accuracy 39.0%, cross entropy 1.833721
I0906 01:05:04.697124 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-67"
I0906 01:05:07.523602 12612 trainer.py:129] Step #68: rate 0.001000, accuracy 32.0%, cross entropy 1.880392
I0906 01:05:07.523602 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-68"
I0906 01:05:10.222730 12612 trainer.py:129] Step #69: rate 0.001000, accuracy 38.0%, cross entropy 1.666129
I0906 01:05:10.223727 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-69"
I0906 01:05:13.138088 12612 trainer.py:129] Step #70: rate 0.001000, accuracy 35.0%, cross entropy 1.829306
I0906 01:05:13.139086 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-70"
I0906 01:05:16.088945 12612 trainer.py:129] Step #71: rate 0.001000, accuracy 38.0%, cross entropy 1.666033
I0906 01:05:16.089943 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-71"
I0906 01:05:18.85739

I0906 01:07:02.580526 12612 trainer.py:129] Step #109: rate 0.001000, accuracy 56.0%, cross entropy 1.343263
I0906 01:07:02.581524 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-109"
I0906 01:07:05.335856 12612 trainer.py:129] Step #110: rate 0.001000, accuracy 53.0%, cross entropy 1.371328
I0906 01:07:05.336891 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-110"
I0906 01:07:08.147092 12612 trainer.py:129] Step #111: rate 0.001000, accuracy 50.0%, cross entropy 1.500600
I0906 01:07:08.148126 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-111"
I0906 01:07:11.078592 12612 trainer.py:129] Step #112: rate 0.001000, accuracy 53.0%, cross entropy 1.447693
I0906 01:07:11.079589 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-112"
I0906 01:07:13.821797 12612 trainer.py:129] Step #113: rate 0.001000, accuracy 54.0%, cross entropy 1.434983
I0906 01:07:13.822794 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-113"
I0906 01:0

I0906 01:08:55.897016 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-150"
I0906 01:08:58.591185 12612 trainer.py:129] Step #151: rate 0.001000, accuracy 60.0%, cross entropy 1.190512
I0906 01:08:58.593180 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-151"
I0906 01:09:01.234588 12612 trainer.py:129] Step #152: rate 0.001000, accuracy 52.0%, cross entropy 1.510713
I0906 01:09:01.235586 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-152"
I0906 01:09:03.961750 12612 trainer.py:129] Step #153: rate 0.001000, accuracy 59.0%, cross entropy 1.171112
I0906 01:09:03.962781 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-153"
I0906 01:09:06.766840 12612 trainer.py:129] Step #154: rate 0.001000, accuracy 63.0%, cross entropy 1.222285
I0906 01:09:06.767836 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-154"
I0906 01:09:09.745658 12612 trainer.py:129] Step #155: rate 0.001000, accuracy 71.0%, cross entropy 0.936193
I0906 01:0

I0906 01:10:50.421043 12612 trainer.py:129] Step #192: rate 0.001000, accuracy 69.0%, cross entropy 0.899482
I0906 01:10:50.422041 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-192"
I0906 01:10:53.057037 12612 trainer.py:129] Step #193: rate 0.001000, accuracy 72.0%, cross entropy 1.056975
I0906 01:10:53.058034 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-193"
I0906 01:10:55.647587 12612 trainer.py:129] Step #194: rate 0.001000, accuracy 67.0%, cross entropy 0.861855
I0906 01:10:55.648584 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-194"
I0906 01:10:58.413877 12612 trainer.py:129] Step #195: rate 0.001000, accuracy 66.0%, cross entropy 1.020816
I0906 01:10:58.414876 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-195"
I0906 01:11:01.336592 12612 trainer.py:129] Step #196: rate 0.001000, accuracy 60.0%, cross entropy 1.243444
I0906 01:11:01.337591 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-196"
I0906 01:1

I0906 01:12:41.086999 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-233"
I0906 01:12:43.770601 12612 trainer.py:129] Step #234: rate 0.001000, accuracy 63.0%, cross entropy 0.972020
I0906 01:12:43.771581 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-234"
I0906 01:12:46.705514 12612 trainer.py:129] Step #235: rate 0.001000, accuracy 66.0%, cross entropy 1.071716
I0906 01:12:46.706512 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-235"
I0906 01:12:49.447813 12612 trainer.py:129] Step #236: rate 0.001000, accuracy 68.0%, cross entropy 0.960652
I0906 01:12:49.448765 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-236"
I0906 01:12:52.171600 12612 trainer.py:129] Step #237: rate 0.001000, accuracy 68.0%, cross entropy 0.961121
I0906 01:12:52.172621 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-237"
I0906 01:12:54.817512 12612 trainer.py:129] Step #238: rate 0.001000, accuracy 74.0%, cross entropy 0.730801
I0906 01:1

I0906 01:14:37.688195 12612 trainer.py:129] Step #275: rate 0.001000, accuracy 72.0%, cross entropy 0.854511
I0906 01:14:37.689193 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-275"
I0906 01:14:40.790040 12612 trainer.py:129] Step #276: rate 0.001000, accuracy 80.0%, cross entropy 0.760763
I0906 01:14:40.790040 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-276"
I0906 01:14:43.545940 12612 trainer.py:129] Step #277: rate 0.001000, accuracy 62.0%, cross entropy 1.182354
I0906 01:14:43.546938 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-277"
I0906 01:14:46.152874 12612 trainer.py:129] Step #278: rate 0.001000, accuracy 74.0%, cross entropy 0.796878
I0906 01:14:46.153871 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-278"
I0906 01:14:48.731030 12612 trainer.py:129] Step #279: rate 0.001000, accuracy 73.0%, cross entropy 0.941776
I0906 01:14:48.732011 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-279"
I0906 01:1

I0906 01:16:34.285607 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-316"
I0906 01:16:37.359227 12612 trainer.py:129] Step #317: rate 0.001000, accuracy 80.0%, cross entropy 0.622850
I0906 01:16:37.360224 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-317"
I0906 01:16:40.299333 12612 trainer.py:129] Step #318: rate 0.001000, accuracy 67.0%, cross entropy 1.074315
I0906 01:16:40.300330 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-318"
I0906 01:16:43.255263 12612 trainer.py:129] Step #319: rate 0.001000, accuracy 66.0%, cross entropy 0.979441
I0906 01:16:43.257257 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-319"
I0906 01:16:46.057101 12612 trainer.py:129] Step #320: rate 0.001000, accuracy 73.0%, cross entropy 1.092968
I0906 01:16:46.058098 12612 trainer.py:175] Saving to "..\logs&checkpoint\conv\ckpt-320"
I0906 01:16:48.671022 12612 trainer.py:129] Step #321: rate 0.001000, accuracy 74.0%, cross entropy 0.837828
I0906 01:1

### Freeze

In [ ]:
save_checkpoint=''

In [ ]:
save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s.pb'%os.path.basename(save_checkpoint))
freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path)